<a href="https://colab.research.google.com/github/elhamod/IS883/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Using `LangChain` for Prompt Engineering

##Q0: Loading OpenAI API key

In [ ]:
!pip install openai==0.28.1

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
config_ini_location = '/content/drive/MyDrive/Colab Notebooks/IS883/OpenAI guide/config.ini' # Change this to point to the location of your config.ini file.

import configparser

config = configparser.ConfigParser()
config.read(config_ini_location)
openai_api_key = config['OpenAI']['API_KEY']

## Q1: FiniBot, financial advising simplified!

As we have seen in class, breaking a prompt down into chains is useful and, if engineered well, returns better and more reliably responses.

[In class](https://github.com/elhamod/IS883/blob/main/Prompt_Engineering_Agents.ipynb), we have seen the most basic form of chains, a `SequentialChain`. However, other more sophisticated chains exist. Here, you will explore a [`RouterChain`](https://python.langchain.com/docs/modules/chains/foundational/router#legacy-routerchain). This chain acts like a switch. Given a condition, it decided which prompt to use. This is in contrast to the deterministic sequantial nature of `SequentialChain`.

Using `RouterChain`, and in a manner similar to [`LangChain` in-class demo](https://github.com/elhamod/IS883/blob/main/Prompt_Engineering_Agents.ipynb), write a solution for the following scenario:

-----------------------------------
You are creating a financial advising ChatBot, FiniBot. Yor clients will fill out [the following csv file](https://docs.google.com/spreadsheets/d/1wJHJE7hrXcedpk9bTQ0oulwTTs41wu32TPTw0FebwXw/edit?usp=sharing), which contains a cell for savings, another for monthly debt, and finally a cell for monthly income. You ChatBot will
 1. Download the file as an `.csv` to your Google Drive.
 2. Load the file to the notebook.
 3. Based on the content of the file, describe your client's situation objectively and in one of two levels: "Novice" and "Expert".
 4. Based on the client's situation:
 - If your client's [debt ratio](https://www.investopedia.com/ask/answers/081214/whats-considered-be-good-debttoincome-dti-ratio.asp#:~:text=Lenders%2C%20including%20anyone%20who%20might,lenders%20prefer%2036%25%20or%20below.) is less than 0.3, then FiniBot will commend the client's financial accomplishment and then turn into an investment advisor. It will advise to invest their money and provide them with an investment portfolio based on their savings and using 5 stocks.
 - Otherwise, FiniBot will politely and cautiously, without taking them on a guilt trip, warn the client about their financial situation. It will then turn into a debt advisor, and create a plan for them to pay off their debt by allocating 10% of their income for monthly debt payments.
---------------------------------------

The final output of your code should be (as text):
---------------------------
<pre>
 - Total savings:  < value >
 - Monthly debt: < value >
 - Monthly income: < value >

 - Financial situation:
 < Your ChatBots Financial summary in "Novice" or "Expert" tone >

- Recommendation:
 < Your advisor's output >
 </pre>
-----------------


In [ ]:
!pip install langchain

Step 1: Load the `.csv` file as text. You can use [`CSVLoader`](https://python.langchain.com/docs/integrations/document_loaders/csv)

In [ ]:
csv_path = "/content/drive/MyDrive/Colab Notebooks/IS883/IS883_Assignment3_FiniBot.csv"

In [ ]:
def loadCSVFile(csv_path):
######
# Fill in your implementation here to return the csv file content as plain text.
#######

  return text

In [ ]:
# printing the content of the file
text = loadCSVFile(csv_path)
print(text)

Here, define `Output_template`, the text that contains the formatted output message as described above. This template has `{input}` as input string. Assume that `{input}` contains the final variables of interest: the input savings, debt, income, and output summary.

In [ ]:
Output_template="""

"""

Here define the `investmet_template`, which should act as the prompt template used when the investment advisor is selected. This template has {input} as input string. Assume that {input} contains the final variables of interest: the input savings, debt, income, and output summary.

In [ ]:
investmet_template ="""

""" + Output_template

Here define the `debt_template`, which should act as the prompt template used when the debt advisor is selected. This template has {input} as input string. Assume that {input} contains the final variables of interest: the input savings, debt, income, and output summary.

In [ ]:
debt_template= """

""" + Output_template

Here, define the two possible routes that can be taken as described above. Follow the example shown [here](https://python.langchain.com/docs/modules/chains/foundational/router#legacy-routerchain).
Each branch should define the **name** of the route, its **description** (which will be used to select the route), and the **template** that will be used for that route.

In [ ]:
routes =

Here, define the variable that sets the `level` of the returned summary.

In [ ]:
level= "Novice" #"Expert"

Create the `langchain.llms.OpenAI` language model. name it `llm`. As you progress with your assignment, consider what appropriate [parameters](https://api.python.langchain.com/en/latest/llms/langchain.llms.openai.OpenAI.html) to use here.

In [ ]:
llm =

**Answer:**


Create a destination chain for each possible route. Follow the example shown [here](https://python.langchain.com/docs/modules/chains/foundational/router#legacy-routerchain).

In [ ]:
destination_chains =

Create a `financial_prompt` for summarizing the client's situation. Include the client's financial knowledge `{level}` as a parameter in this template.
- Make sure to use the prompt engineering tips we used in class. You may find [these tips](https://pbs.twimg.com/media/F0RWXIjXgAARNAG?format=jpg&name=4096x4096) useful.
- Make sure your prompt is talking the client as a chatbot, and not in 3rd person.

In [ ]:
financial_prompt = """

"""

This template is needed by `RouterChain`.

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """\
Given a raw text input to a language model select the model prompt best suited for \
the input. You will be given the names of the available prompts and a description of \
what the prompt is best suited for. You may also revise the original input if you \
think that revising it will ultimately lead to a better response from the language \
model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \\ name of the prompt to use or "DEFAULT"
    "next_inputs": string \\ A stringified JSON. Do not include the original input. It contains the following keys: the "savings" value, the "debt" value, the "income" value, and the "summary" provided above.
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR \
it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: Make sure that "next_inputs" is not a string. This is very important!


<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the response) >>
<< OUTPUT (must end with ```) >>
"""

The following code creates the routing template

In [ ]:
prompt = financial_prompt + MULTI_PROMPT_ROUTER_TEMPLATE

In [ ]:
destinations = [f"{route['name']}: {route['description']}" for route in routes]
destinations_str = "\n".join(destinations)
router_template = prompt.format(destinations=destinations_str, level=level)

In [ ]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains import ConversationChain

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=ConversationChain(llm=llm, output_key="text"),
    verbose=False,
)

Now, try it! Write a code that:
- Turns `langchain.debug` on so you could see what it going on.
- Runs the `chain` we created above.

Answer the following questions:



1.   Iteratively improve your code and prompts until you achieve the desired output.

  - Does the final output match expectation? If not, what are the "bugs" from the client's perspective and what's your justification for their occurance?
  - What "extra mile" you could have gone to improve results?










**Answer:**


2. With the final chain, for each of the following scenarios, run the `chain` 10 times to sample the spectrum of generated answers:

*   A `csv` where the debt is `$500` and the savings are `$4,000`, with a monthly income of `$4,000`
*   A `csv` where the debt is `$500` and the savings are `$0`, with a monthly income of `$4,000`
*   A `csv` where the debt is `$1,500` and the savings are `$4,000`, with an monthly income of `$3,000`



In [ ]:
def run10times(csv_file):



In [ ]:
## Call for first scenario

In [ ]:
## Call for second scenario

In [ ]:
## Call for third scenario

- State the expected correct output for each scenario in your own words.
- What was the success rate for each scenario (as a percentage)? How did you calculate it?
- Can you explain the low or high rate of success for each scenario? Comment, with suggestions when appropriate, on your findings.

**Answer:**


##Q2 (BONUS): FiniBot is online!

You are now ready to start offering FiniBot services to the public! Here is what you need to do:

Create a User interface using `streamlit` that consists of:
- An `st.header` containing text welcoming your client and asking them to upload the spreadsheet. Provide a link to the spreadsheet's template in this header.
- An [`st.file_uploader`](https://docs.streamlit.io/library/api-reference/widgets/st.file_uploader) "Upload spreadsheet" button. [This](https://stackoverflow.com/questions/68248125/how-to-read-csv-file-from-user-in-streamlit-and-converting-to-pandas-dataframe) may also be useful.
- An [`st.radio`](https://docs.streamlit.io/library/api-reference/widgets/st.radio) toggle button to select whether the client is a "Novice" or an "Expert".
- Displays the spreadsheet neatly, like [this](https://stackoverflow.com/questions/68248125/how-to-read-csv-file-from-user-in-streamlit-and-converting-to-pandas-dataframe).
- An `st.markdown` to output FiniBot's analysis and recommendation.

You may find [this cheatsheet](https://cheat-sheet.streamlit.app/) helpful while you are constructing your UI.

To create the app, follow these steps:

1. Fork the [following GitHub](https://github.com/elhamod/IS883_Azure_OpenAI_demo/tree/Assignment3). You will be filling your code inside `app.py`
2. Develop `streamlit`'s code as described above. You do not need to have language modeling at this point yet. You only need to have a functioning UX.
3. Copy the code you have developed in Q1 to `app.py` and integrate it with `streamlit`'s code ***appropriately***. You will have to make some adjustments (e.g., getting the level from the toggle button rather than hard-coding it).
4. Commit, push, and deploy!

Here are some guides for working with Streamlit's community cloud:

- For getting started and signing up (please follow only the first 8 steps): [Streamlit Community Cloud Quickstart](https://docs.streamlit.io/streamlit-community-cloud/get-started/quickstart)
- For deploying from your GitHub repository: [Deploying to Streamlit Cloud](https://docs.streamlit.io/streamlit-community-cloud/deploy-your-app)
- Make sure to add `OPENAI_API_KEY` in advanced settings